# Model evaluation notebook

in this notebook we will evaluate % availability. definitions are:

X = 
    location: site_id
    time: hour, dow, weekend, is_holiday, 
    weather: temperature, dew point, relative humidity, wind direction, wind speed
    grid: net demand, solar, wind
y = % available (for site)

this will work for all sites.

How will we do train test split?
random 80-20 for now 


In [1]:
import os
if 'models' == os.getcwd().split('/')[-1]: os.chdir('..')
if 'ev_charging' == os.getcwd().split('/')[-1]: print('in the right place!')
else: os.chdir('/Users/varunvenkatesh/Documents/Github/ev_charging')
os.getcwd()

in the right place!


'/home/denny/Documents/mids/ev_charging'

In [2]:
import pandas as pd
from src.weather import get_processed_hourly_7day_weather


In [3]:
# hist_weather = pd.read_csv('./data/caltech_historical_weather')

In [4]:
historical_weather = pd.DataFrame({})

In [5]:
weather_features = ['temperature_degC', 'dewpoint_degC','relative_humidity_%', 'wind_speed_mph']
hist_weather = pd.read_csv('./data/caltech_historical_weather.csv').set_index('time')[weather_features]
hist_weather['site'] = 'Caltech'
if historical_weather.empty: 
    historical_weather = hist_weather
else:
    historical_weather = pd.concat([historical_weather, hist_weather], axis=0)

In [6]:
weather_features = ['temperature_degC', 'dewpoint_degC','relative_humidity_%', 'wind_speed_mph']
hist_weather = pd.read_csv('./data/caltech_historical_weather.csv').set_index('time')[weather_features]
hist_weather['site'] = 'JPL'
if historical_weather.empty: 
    historical_weather = hist_weather
else:
    historical_weather = pd.concat([historical_weather, hist_weather], axis=0)

In [7]:
weather_features = ['temperature_degC', 'dewpoint_degC','relative_humidity_%', 'wind_speed_mph']
hist_weather = pd.read_csv('./data/office_historical_weather.csv').set_index('time')[weather_features]
hist_weather['site'] = 'Office001'
if historical_weather.empty: 
    historical_weather = hist_weather
else:
    historical_weather = pd.concat([historical_weather, hist_weather], axis=0)

In [8]:
historical_weather.index = pd.to_datetime(historical_weather.index, utc=True)

In [9]:
historical_weather.site.unique()

array(['Caltech', 'JPL', 'Office001'], dtype=object)

In [10]:
from src.weather import get_processed_hourly_7day_weather
site2latlon = {'Caltech':(34.134785646454844, -118.11691382579643),
               'Office001':(37.33680466796926, -121.90743423142634),
               'JPL':(34.20142342818471, -118.17126565774107)}
future_weather = get_processed_hourly_7day_weather(*site2latlon['Caltech'])

https://api.weather.gov/points/34.134785646454844,-118.11691382579643
status code: 200
https://api.weather.gov/gridpoints/LOX/160,48/forecast/hourly


In [11]:
future_weather.to_csv('data/test_future_weather.csv')

 confirm that past and future weather have the same features

In [12]:
print(f'future weather features are \n{list(future_weather[weather_features].columns)}, \nand historical weather features are \n{list(hist_weather[weather_features].columns)}')

future weather features are 
['temperature_degC', 'dewpoint_degC', 'relative_humidity_%', 'wind_speed_mph'], 
and historical weather features are 
['temperature_degC', 'dewpoint_degC', 'relative_humidity_%', 'wind_speed_mph']


# get caiso data

In [13]:
from src import oasis
import datetime
import pandas as pd
from src.oasis import SystemDemand
import urllib
import datetime
sd = SystemDemand()

In [14]:
start = datetime.datetime(2024,4,8)
end = datetime.datetime(2024,4,9)
demand_forecast = sd.get_demand_forecast(start, end)

# Get historical demand

the data from the request is the bytes for a zip file. to get it to a pandas dataframe,
1. read the content with the io.BytesIO
2. open the bytes as a zip file with ZipFile
3. open the zip file by getting the first (and only) name in the extraction
4. read the data using the .read() method of the file object
5. convert the raw data to a string with BytesIO again
6. read the bytes to a dataframe with the pd.read_csv file

In [15]:
historical_caiso_actual_load = pd.read_csv('data/historical_caiso_actual_load.csv', index_col=[0])

In [16]:
TACS = ['SCE-TAC', 'PGE-TAC']
MARKET_RUN_IDS = ['ACTUAL']

In [17]:
historical_caiso_actual_load['datetime'] = pd.to_datetime(historical_caiso_actual_load['OPR_DT']) + pd.to_timedelta(historical_caiso_actual_load['OPR_HR'], unit='h')
historical_caiso_actual_load = historical_caiso_actual_load[
    (historical_caiso_actual_load['TAC_AREA_NAME'].isin(TACS)) &
    (historical_caiso_actual_load['MARKET_RUN_ID'].isin(MARKET_RUN_IDS))
]

In [18]:

demand_forecast['datetime'] = pd.to_datetime(demand_forecast['OPR_DT']) + pd.to_timedelta(demand_forecast['OPR_HR'], unit='h')
demand_forecast = demand_forecast[
    (demand_forecast['TAC_AREA_NAME'].isin(TACS)) &
    (demand_forecast['MARKET_RUN_ID'].isin(MARKET_RUN_IDS))
]

In [19]:
demand_features = ['datetime', 'TAC_AREA_NAME', 'MARKET_RUN_ID', 'MW']

In [20]:
demand_forecast[demand_features].columns == historical_caiso_actual_load[demand_features].columns

array([ True,  True,  True,  True])

Modeling

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

from datetime import datetime

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

from src.data_preprocessing import  datetime_processing, userinput_processing, holiday_processing, create_all_site_x, create_all_site_y, create_x, update_varuns_x, get_start_end_times

In [22]:
df_c = pd.read_parquet('data/sessions/caltech/').reset_index(drop=True)
df_c = datetime_processing(df_c)
df_c = userinput_processing(df_c)

df_j = pd.read_parquet('data/sessions/jpl/').reset_index(drop=True)
df_j = datetime_processing(df_j)
df_j = userinput_processing(df_j)

df_o = pd.read_parquet('data/sessions/office001/').reset_index(drop=True)
df_o = datetime_processing(df_o)
df_o = userinput_processing(df_o)

cols = list(df_c.columns)
df = pd.concat([df_c[cols], df_j[cols], df_o[cols]], axis=0)

In [23]:
print(f'df is a dataframe comprised of these site ids: {list(df.siteID.unique())}')

df is a dataframe comprised of these site ids: [2, 1, 19]


# Make X and y

X contains 4 different sets of data - location, datetime, weather, energy demand

In [24]:
X = pd.DataFrame()
sites_df = [df_o,df_c,df_j]
sites = ['Office001','Caltech','JPL']
site_ids = [2,1,19]
site2id = { k:v for (k,v) in zip(sites, site_ids)}
site2tac = {2:'PGE-TAC',
            1:'SCE-TAC',
            19:'SCE-TAC',
            }
    

In [43]:
X = pd.DataFrame()
for sdf, id, site_name in zip(sites_df, site_ids, sites):
    start, end = get_start_end_times(sdf)
    print(site_name, start, end)
    # time
    x = pd.DataFrame(index=pd.date_range(start, end, inclusive='both', freq='h', tz=0),
                     columns=['dow', 'hour', 'month', 'is_sunny'])
    x['dow'] = x.index.dayofweek
    x['hour'] = x.index.hour
    x['month'] = x.index.month
    x['is_sunny'] = 0
    
    x = holiday_processing(x)
    
    print('x rows: ', x.shape[0])
    # energy demand
    demand = historical_caiso_actual_load[demand_features].set_index('datetime').rename(columns={'MW':'actual_demand_MW'}).sort_index()
    demand = demand[demand['TAC_AREA_NAME'] == site2tac[id]][['actual_demand_MW']]
    demand.index = pd.to_datetime(demand.index, utc=True)
    demand = demand.loc[x.index.min():x.index.max(),:]
    print('dem rows', demand.shape[0])
    
    x = pd.merge(x,demand, left_index=True, right_index=True, how='left')
    
    # weather and location
    weather = historical_weather[historical_weather['site']==site_name]
    weather = weather.loc[x.index.min():x.index.max(), :]
    print('weather rows:', weather.shape[0])
    x = pd.merge(x, weather, left_index=True, right_index=True, how='left')
    print('together rows: ', x.shape[0])
    if X.empty:
        X = x
    else:
        X = pd.concat([X, x], axis=0)

X

Office001 2019-03-25 2021-09-12
x rows:  21649
dem rows 8593
weather rows: 21649
together rows:  21650
Caltech 2018-05-01 2021-09-14
x rows:  29569
dem rows 16465
weather rows: 29569
together rows:  29571
JPL 2018-09-05 2021-09-13
x rows:  26497
dem rows 13417
weather rows: 26497
together rows:  26499


,dow,hour,month,is_sunny,is_holiday,actual_demand_MW,temperature_degC,dewpoint_degC,relative_humidity_%,wind_speed_mph,site
2019-03-25 00:00:00+00:00,0,0,3,0,False,9470.0,18.9,-1.6,25.0,9.20,Office001
2019-03-25 01:00:00+00:00,0,1,3,0,False,9036.0,19.4,-1.7,24.0,6.96,Office001
2019-03-25 02:00:00+00:00,0,2,3,0,False,8963.0,15.0,-2.0,31.0,14.98,Office001
2019-03-25 03:00:00+00:00,0,3,3,0,False,8818.0,13.3,-2.2,34.0,10.31,Office001
2019-03-25 04:00:00+00:00,0,4,3,0,False,8840.0,13.3,-1.9,35.0,0.00,Office001
...,...,...,...,...,...,...,...,...,...,...,...
2021-09-12 20:00:00+00:00,6,20,9,0,False,NaN,33.4,8.8,22.0,5.84,JPL
2021-09-12 21:00:00+00:00,6,21,9,0,False,NaN,35.7,0.7,11.0,12.74,JPL
2021-09-12 22:00:00+00:00,6,22,9,0,False,NaN,34.5,-2.9,9.0,12.74,JPL
2021-09-12 23:00:00+00:00,6,23,9,0,False,NaN,34.5,-0.2,11.0,10.31,JPL


In [46]:
X['site'].reset_index().drop_duplicates()

,index,site
0,2019-03-25 00:00:00+00:00,Office001
1,2019-03-25 01:00:00+00:00,Office001
2,2019-03-25 02:00:00+00:00,Office001
3,2019-03-25 03:00:00+00:00,Office001
4,2019-03-25 04:00:00+00:00,Office001
...,...,...
77715,2021-09-12 20:00:00+00:00,JPL
77716,2021-09-12 21:00:00+00:00,JPL
77717,2021-09-12 22:00:00+00:00,JPL
77718,2021-09-12 23:00:00+00:00,JPL


In [42]:
X[X['site'] == 'JPL']

,dow,hour,month,is_sunny,is_holiday,actual_demand_MW,temperature_degC,dewpoint_degC,relative_humidity_%,wind_speed_mph,site
2018-09-05 00:00:00+00:00,2,0,9,0,False,12307.0,23.4,17.8,71.0,11.43,JPL
2018-09-05 01:00:00+00:00,2,1,9,0,False,11541.0,21.8,17.4,76.0,11.43,JPL
2018-09-05 02:00:00+00:00,2,2,9,0,False,11014.0,20.7,16.7,78.0,10.31,JPL
2018-09-05 03:00:00+00:00,2,3,9,0,False,10628.0,18.4,16.7,90.0,8.08,JPL
2018-09-05 04:00:00+00:00,2,4,9,0,False,10440.0,18.4,17.2,93.0,8.08,JPL
...,...,...,...,...,...,...,...,...,...,...,...
2021-09-12 20:00:00+00:00,6,20,9,0,False,NaN,33.4,8.8,22.0,5.84,JPL
2021-09-12 21:00:00+00:00,6,21,9,0,False,NaN,35.7,0.7,11.0,12.74,JPL
2021-09-12 22:00:00+00:00,6,22,9,0,False,NaN,34.5,-2.9,9.0,12.74,JPL
2021-09-12 23:00:00+00:00,6,23,9,0,False,NaN,34.5,-0.2,11.0,10.31,JPL


In [27]:
X['site'].value_counts().sum() == X.shape[0]
print(f'there are {X.shape[0]} records')

there are 77720 records


In [31]:
X['site'].value_counts()

site
Caltech      29571
JPL          26499
Office001    21650
Name: count, dtype: int64

In [33]:
y

2019-03-25 00:00:00+00:00    1.000000
2019-03-25 01:00:00+00:00    1.000000
2019-03-25 02:00:00+00:00    1.000000
2019-03-25 03:00:00+00:00    1.000000
2019-03-25 04:00:00+00:00    1.000000
                               ...   
2021-09-12 20:00:00+00:00    0.961538
2021-09-12 21:00:00+00:00    0.980769
2021-09-12 22:00:00+00:00    1.000000
2021-09-12 23:00:00+00:00    1.000000
2021-09-13 00:00:00+00:00    1.000000
Length: 77715, dtype: float64

# Deal with covid
we found (graphically) that covid effects start around march 3rd 2020, so we truncated our data to march 1st. Since we did not have dates that contained post covid data, we did not include any charges after 2020-03-01 

In [28]:
x = X[X.index < '2020-03-01']
X.index.max()


Timestamp('2021-09-14 00:00:00+0000', tz='UTC')

In [29]:
cols = list(df_c.columns)
df = pd.concat([df[cols] for df in sites_df], axis=0)
y = create_all_site_y(df, regression=True)

In [30]:
assert X.shape[0] == y.shape[0], 'X and y rows are not equal!'

AssertionError: X and y rows are not equal!

In [34]:
y = y[y.index < '2020-03-01']
y.index.max()

Timestamp('2020-02-29 23:00:00+0000', tz='UTC')

In [35]:
assert x.shape[0] == y.shape[0], 'forgot to trim y\'s covid!'

AssertionError: forgot to trim y's covid!

# XGBoost

In [ ]:
X.columns

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = .2)

cat_cols = ['dow', 'hour', 'month', 'is_sunny', 'is_holiday','siteID']
num_cols = [col for col in x.columns if col not in cat_cols]

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')


xgb_model = Pipeline(
    steps=[
        ('preprocessing', ColumnTransformer(
            transformers=[
                ("num", numeric_transformer, num_cols),
                ("cat", categorical_transformer, cat_cols),
            ])),
        ('xgb', XGBRegressor()),
        ]
)

xgb_model.fit(X_train, y_train)
print(f'training RMSE: {np.sqrt(mean_squared_error(xgb_model.predict(X_train), y_train))}\ntest RMSE    : {np.sqrt(mean_squared_error(xgb_model.predict(X_test), y_test))}')
print(f'training r2: {xgb_model.score(X_train, y_train)}\ntest r2    : {xgb_model.score(X_test, y_test)}')

In [ ]:
X_test = X_test.sort_index()
y_test = y_test.sort_index()
pred = pd.Series(xgb_model.predict(X_test), index=X_test.index)
plt.title('XGB Regression model')
plt.suptitle(f'test R2: {np.round(xgb_model.score(X_test, y_test),3)}')
plt.plot(y_test.index, y_test)
plt.plot(y_test.index, pred)
plt.ylabel('% available')